# Building a LangChain based app to retrieve Wikipedia Embeddings

Cohere [recently embedded](https://txt.cohere.com/embedding-archives-wikipedia/) the entire wikipedia and made it available via [HuggingFace](https://huggingface.co/Cohere?ref=txt.cohere.com). Weviate hosted the embeddings and made it available for [free](https://weaviate.io/developers/weaviate/more-resources/example-datasets#semantic-search-through-wikipedia)

Below is the code to retrieve from the Weviate Vector Database

### Code to retrive using semantic search

In [32]:
import os
import weaviate
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Weaviate
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
cohere_api_key = os.getenv('COHERE_API_KEY')
weaviate_api_key = os.getenv('weaviate_api_key')
weaviate_url = os.getenv('weaviate_url')


# Connect to the Weaviate demo databse containing 10M wikipedia vectors
# This uses a public READ-ONLY Weaviate API key
auth_config = weaviate.auth.AuthApiKey(api_key=weaviate_api_key) 

client = weaviate.Client( url=weaviate_url, auth_client_secret=auth_config, 
                         additional_headers={ "X-Cohere-Api-Key": cohere_api_key})


vectorstore = Weaviate(client,  index_name="Articles", text_key="text")
vectorstore._query_attrs = ["text", "title", "url", "views", "lang", "_additional {distance}"]
vectorstore.embedding =CohereEmbeddings(model="embed-multilingual-v2.0", cohere_api_key=cohere_api_key)
llm =OpenAI(temperature=0, openai_api_key=openai_api_key)
qa = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever())
query = "Why is the theory of everything significant?"
result = qa({"query": query})

In [20]:
result['result']

' La teoría del todo es significativa porque podría unificar todas las interacciones fundamentales de la naturaleza, que son consideradas como cuatro: gravitación, la fuerza nuclear fuerte, la fuerza nuclear débil y la electromagnética. Esto permitiría una comprensión profunda de varios tipos diferentes de partículas, así como de diferentes fuerzas.'

Great the retrieval works, but do we know if the most relevant results are being sent for answer generation. Also, the answer is not in English, can we fix it? 

Let's see what happening under the hood.

#### Both methods below help retrive Documents based on similarity search

In [37]:
query = "Why is the theory of everything significant?"
docs = vectorstore.similarity_search(query, 10)
docs

[Document(page_content='सर्वतत्व सिद्धांत या हर चीज़ का सिद्धांत या सब कुछ का सिद्धांत () सैद्धांतिक भौतिकी का एक कल्पित सिद्धांत है जो हमारे भौतिक ब्रह्माण्ड में घट सकने वाली हर चीज़ को वैज्ञानिक दृष्टि से समझाने की क्षमता रखता होगा। अगर यह सिद्धांत स्पष्ट हो जाता है तो ऐसा कोई भी प्रयोग नहीं होगा जिसके नतीजे के बारे में पहले से ही सही भविष्यवाणी करनी सम्भव न हो। यह सिद्धांत अन्य सभी ज्ञात सिद्धांतो को एक-दूसरे से जोड़ने का भी काम करेगा। वैज्ञानिकों द्वारा सर्वतत्व सिद्धांत ढूंढने का एक मुख्य कारण प्रमात्रा यान्त्रिकी (क्वान्टम मकैनिक्स) और सामान्य सापेक्षता (थीओरी ऑफ़ रॅलॅटिविटि) के बीच में तालमेल बनाना है। बिना इस सिद्धांत के मिले यह दोनों मूल सिद्धांत कुछ पहलूओं में एक-दूसरे का खंडन करते हैं।', metadata={'_additional': {'distance': -150.66002}, 'lang': 'hi', 'title': 'सर्वतत्व सिद्धांत', 'url': 'https://hi.wikipedia.org/wiki?curid=514328', 'views': 40}),
 Document(page_content='Una teoría del todo (o ToE por sus siglas en inglés, "Theory of Everything") es una teoría hipotética de 

In [38]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
docs1 = retriever.get_relevant_documents(query)
docs1

[Document(page_content='सर्वतत्व सिद्धांत या हर चीज़ का सिद्धांत या सब कुछ का सिद्धांत () सैद्धांतिक भौतिकी का एक कल्पित सिद्धांत है जो हमारे भौतिक ब्रह्माण्ड में घट सकने वाली हर चीज़ को वैज्ञानिक दृष्टि से समझाने की क्षमता रखता होगा। अगर यह सिद्धांत स्पष्ट हो जाता है तो ऐसा कोई भी प्रयोग नहीं होगा जिसके नतीजे के बारे में पहले से ही सही भविष्यवाणी करनी सम्भव न हो। यह सिद्धांत अन्य सभी ज्ञात सिद्धांतो को एक-दूसरे से जोड़ने का भी काम करेगा। वैज्ञानिकों द्वारा सर्वतत्व सिद्धांत ढूंढने का एक मुख्य कारण प्रमात्रा यान्त्रिकी (क्वान्टम मकैनिक्स) और सामान्य सापेक्षता (थीओरी ऑफ़ रॅलॅटिविटि) के बीच में तालमेल बनाना है। बिना इस सिद्धांत के मिले यह दोनों मूल सिद्धांत कुछ पहलूओं में एक-दूसरे का खंडन करते हैं।', metadata={'_additional': {'distance': -150.66002}, 'lang': 'hi', 'title': 'सर्वतत्व सिद्धांत', 'url': 'https://hi.wikipedia.org/wiki?curid=514328', 'views': 40}),
 Document(page_content='Una teoría del todo (o ToE por sus siglas en inglés, "Theory of Everything") es una teoría hipotética de 

These are queite many different results from different language wikipedia articles. Can we find out which answers are most relavant?

### Contextual Compression helps pass only the most relavant answers to the LLM for answer completion

In [36]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

retriever = vectorstore.as_retriever(search_kwargs={"k": 15})
compressor = CohereRerank(model='rerank-multilingual-v2.0', top_n=4 )
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)
compressed_docs = compression_retriever.get_relevant_documents("Why is the theory of everything significant?")
compressed_docs

[Document(page_content='في الفيزياء الحالية السائدة، نظرية كل شيء هي محاولة لتوحيد القوى الأساسية الأربعة الموجودة في الطبيعة: (أي الثقالة, و القوة النووية القوية, و القوة النووية الضعيفة, و القوة الكهرومغناطيسية). بما أن القوة الضعيفة لها القدرة على تغيير الجسيمات الأولية من شكل لآخر، سينبغي على نظرية كل شيء بأن تعطينا فهماً عميقا للعلاقات الموجودة بين جميع الجسيمات المختلفة،', metadata={'_additional': {'distance': -149.68802}, 'lang': 'ar', 'title': 'نظرية كل شيء', 'url': 'https://ar.wikipedia.org/wiki?curid=10751', 'views': 200, 'relevance_score': 0.9994207}),
 Document(page_content="Nella corrente principale dell'attuale fisica, una Teoria del Tutto unificherebbe tutte le interazioni fondamentali della natura, che sono solitamente considerate essere quattro in numero: gravità, forza nucleare forte, forza nucleare debole e forza elettromagnetica. Siccome la forza debole può trasformare le particelle elementari da un tipo a un altro, la Teoria del Tutto dovrebbe dare una profonda com

As you can see, there is a 'relevance_score' now added to the metadata of each Document retrived, this can ensure the most relavant Documents are used for the next step of answer generation via an LLM call. 

In [24]:
qa = RetrievalQA.from_chain_type(llm, retriever=compression_retriever)
result = qa({"query": query})

In [25]:
result['result']

' La teoria del tutto è significativa perché cerca di unificare tutte le interazioni fondamentali della natura in un unico quadro teorico.'

Now the answer is still in a language different than English. There are many ways to fix this. An easy method is to edit the prompt_template to add a language parameter

In [26]:
from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer in {language}:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question","language"])

In [28]:
qa = RetrievalQA.from_chain_type(llm, retriever=compression_retriever, chain_type_kwargs={"prompt": PROMPT.partial(language="english")})
result = qa({"query": query})
result['result']

' The theory of everything is significant because it would attempt to unify the four fundamental forces of nature: gravity, strong nuclear force, weak nuclear force, and electromagnetism. It would also give us a deeper understanding of the relationships between different particles and forces.'

Great! Now we have a response in English and also from the most relavant sources from global Wikipedia articles. 